# Динамика показателей канала дома и на даче
Пример расчета показателей телеканалов в разбивке по месту просмотра.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 31.03.2022 по месяцам + тотал за период
- Временной интервал: 05:00-29:00
- ЦА: Все 14-44
- Место просмотра: Все места с разбивкой на места
- Каналы: ТНТ									
- Статистики: Share, Rtg(000)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ТНТ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

Запрошены записи: 0 - 125
Всего найдено записей: 125



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
120,2,10,Возраст,Age,10
121,2,89,Возраст,Age,89
122,2,8,Возраст,Age,8
123,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


In [4]:
# Справочник мест просмотра
cats.get_tv_location()

Запрошены записи: 0 - 2
Всего найдено записей: 2



,id,name,ename
0,2,Дача,Dacha
1,1,Дом,Home


## Формирование задания
Зададим условия расчета

In [5]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-01-01', '2022-03-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 14 AND age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (83)'

# Указываем список статистик для расчета
statistics = ['Share', 'Rtg000']

# Указываем срезы
slices = ['researchMonth']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Зададим группу требуемых мест просмотра

In [6]:
locs = {
    # Если в фильтре не задавать location_filter, то расчет будет сделан по дом+дача
    "Все места": {"location": None},
    # Дача
    "Дача": {"location": "locationId = 2"},
    # Дом
    "Дом": {"location": "locationId = 1"}
}

## Расчет задания

In [7]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы срезов формируем задания и отправляем на расчет
for loc, logical in locs.items():
    
    # Формируем задание на расчет
    project_name = f"{loc}" 
    location_filter = logical['location']
          
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_months = pd.concat(results)
df_months

Отправляем задания на расчет
...
Ждем выполнения
Расчет задач (3) [ = = = = = = = = = = = = = = = ] время расчета: 0:00:52.759698
Расчет завершен, получаем результат
Собираем таблицу
...

,prj_name,researchMonth,Share,Rtg000
0,Все места,2022-01-01,7.295942,462.622185
1,Все места,2022-02-01,7.302614,465.210411
2,Все места,2022-03-01,6.896560,419.159924
0,Дача,2022-01-01,5.315849,11.261132
1,Дача,2022-02-01,4.731991,8.110157
2,Дача,2022-03-01,4.419731,7.092483
0,Дом,2022-01-01,7.331783,451.361053
1,Дом,2022-02-01,7.347883,457.100253
2,Дом,2022-03-01,6.941480,412.067442


Теперь посчитаем тотал значение за весь период

In [8]:
# для этого уберем разбивку по месяцам - оставим параметр slices пустым
slices = []

Прогоним цикл еще раз

In [9]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы срезов формируем задания и отправляем на расчет
for loc, logical in locs.items():
    
    # Формируем задание на расчет
    project_name = f"{loc}" 
    location_filter = logical['location']
          
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_total = pd.concat(results)
df_total

Отправляем задания на расчет
...
Ждем выполнения
Расчет задач (3) [ = = = = = = = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:29.954712
Расчет завершен, получаем результат
Собираем таблицу
...

,prj_name,Share,Rtg000
0,Все места,7.164481,448.457077
0,Дача,4.871572,8.844961
0,Дом,7.206073,439.612116


In [10]:
# добавим колонку с периодом
df_total.insert(1, 'researchMonth', 'total_period')

Объединяем две таблицы в одну

In [11]:
df = pd.concat([df_months, df_total])
df

,prj_name,researchMonth,Share,Rtg000
0,Все места,2022-01-01,7.295942,462.622185
1,Все места,2022-02-01,7.302614,465.210411
2,Все места,2022-03-01,6.896560,419.159924
0,Дача,2022-01-01,5.315849,11.261132
1,Дача,2022-02-01,4.731991,8.110157
2,Дача,2022-03-01,4.419731,7.092483
0,Дом,2022-01-01,7.331783,451.361053
1,Дом,2022-02-01,7.347883,457.100253
2,Дом,2022-03-01,6.941480,412.067442
0,Все места,total_period,7.164481,448.457077


Настраиваем внешний вид таблицы

In [12]:
df_res = pd.pivot_table(df, values=['Share', 'Rtg000'],
                        index=['researchMonth'], 
                        columns=['prj_name'])
df_res

Rtg000                            Share                    
prj_name        Все места       Дача         Дом Все места      Дача       Дом
researchMonth                                                                 
2022-01-01     462.622185  11.261132  451.361053  7.295942  5.315849  7.331783
2022-02-01     465.210411   8.110157  457.100253  7.302614  4.731991  7.347883
2022-03-01     419.159924   7.092483  412.067442  6.896560  4.419731  6.941480
total_period   448.457077   8.844961  439.612116  7.164481  4.871572  7.206073

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [13]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_locations_by_months'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()